# SEC EDGAR Agent with LangChain (Python): AI-Powered Financial Analysis

Turn SEC filings into investment insights in seconds using LangChain agents.

## What This Does
- Automatically pulls SEC filings (10-K, 10-Q, 8-K)
- Extracts financial metrics and compares year-over-year
- Analyzes insider trading patterns
- Identifies material events and risks
- Generates investment insights with specific data points

## Installation

In [ ]:
# Install required packages
!pip install langchain langchain-openai langchain-core pydantic requests python-dateutil

## Setup

In [ ]:
import os
import json
from datetime import datetime, timedelta

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your actual key

# Or uncomment to load from .env file
# from dotenv import load_dotenv
# load_dotenv()

## Initialize SEC EDGAR Toolkit

First, let's set up our SEC EDGAR toolkit with all available tools.

In [ ]:
from sec_edgar_langchain import SECEdgarToolkit, SECEdgarConfig
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

# Initialize the SEC EDGAR toolkit
config = SECEdgarConfig(
    user_agent="FinancialAnalystDemo/1.0 (demo@example.com)"  # Required by SEC
)

toolkit = SECEdgarToolkit(config=config)
tools = toolkit.get_tools()

print(f"✅ Loaded {len(tools)} SEC EDGAR tools:")
for tool in tools:
    print(f"  📊 {tool.name}")

## Create the Financial Analyst Agent

In [ ]:
# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4-turbo-preview",
    temperature=0,
    streaming=True
)

# Create the ReAct prompt
prompt = PromptTemplate.from_template("""
You are a Senior Financial Analyst with expertise in SEC filings analysis.
You have access to real-time SEC EDGAR data through various tools.

IMPORTANT Guidelines:
1. Always start by looking up the company's CIK number using sec_edgar_cik_lookup
2. Use specific tools to get accurate data - don't make up numbers
3. Cite specific filing dates and form types when referencing data
4. Provide quantitative analysis with actual numbers
5. For financial data, check company facts and recent filings

Answer the following question as best you can using the available tools:

Question: {input}

You have access to the following tools:
{tools}

Use the following format:
Thought: Consider what information you need to answer the question
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now have enough information to answer the question
Final Answer: the final answer to the original question with specific data points

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")

# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print("🤖 Financial Analyst Agent ready!")

## Demo 1: Comprehensive Earnings Analysis

Let's analyze NVIDIA's recent performance - perfect timing with the AI boom!

In [ ]:
# Analyze NVIDIA's earnings
query = """
Analyze NVIDIA (NVDA) latest earnings:
1. What was their most recent quarterly revenue?
2. How does it compare to the same quarter last year?
3. What's their net income trend?
4. Any recent material events from 8-K filings?
"""

print("🔍 Analyzing NVIDIA earnings...\n")
result = agent_executor.invoke({"input": query})
print("\n" + "="*60)
print("📈 ANALYSIS RESULTS:")
print("="*60)
print(result["output"])

## Demo 2: Insider Trading Analysis

Are insiders buying or selling? Let's check Tesla!

In [ ]:
# Check Tesla insider trading
insider_query = """
Analyze Tesla (TSLA) insider trading activity:
1. How many Form 4 filings in the last 30 days?
2. Are executives buying or selling?
3. What's the sentiment - bullish or bearish?
"""

print("💼 Checking Tesla insider activity...\n")
insider_result = agent_executor.invoke({"input": insider_query})
print("\n📊 Insider Trading Analysis:")
print(insider_result["output"])

## Demo 3: Competitive Analysis

Compare tech giants head-to-head!

In [ ]:
# Compare Apple vs Microsoft
comparison_query = """
Compare Apple (AAPL) vs Microsoft (MSFT):
1. Latest quarterly revenues for each
2. Net income comparison
3. Which has better profit margins?
4. Recent 8-K events for both
Which looks stronger for next quarter?
"""

print("⚔️ Comparing Apple vs Microsoft...\n")
comparison_result = agent_executor.invoke({"input": comparison_query})
print("\n🏆 Competitive Analysis:")
print(comparison_result["output"])

## Demo 4: Material Events Scanner

Catch market-moving events before everyone else!

In [ ]:
# Scan for recent material events
events_query = """
Check Amazon (AMZN) for recent material events:
1. What 8-K filings were filed in the last 30 days?
2. Any acquisitions, leadership changes, or strategic shifts?
3. Rate the significance of these events
"""

print("🚨 Scanning for material events...\n")
events_result = agent_executor.invoke({"input": events_query})
print("\n📰 Material Events:")
print(events_result["output"])

## Build Your Own Analysis Function

Create a reusable earnings prep function!

In [ ]:
def analyze_earnings(ticker: str, company_name: str):
    """
    Automated earnings analysis for any company.
    
    Args:
        ticker: Stock ticker symbol
        company_name: Company name
    
    Returns:
        Dict with analysis results
    """
    query = f"""
    Prepare comprehensive earnings analysis for {company_name} ({ticker}):
    
    1. FINANCIAL PERFORMANCE:
       - Latest quarterly revenue and net income
       - Year-over-year growth rates
       - EPS if available
    
    2. RECENT DEVELOPMENTS:
       - Any 8-K filings in last 30 days
       - Material events or announcements
    
    3. INSIDER SENTIMENT:
       - Form 4 filings in last 30 days
       - Net buying or selling by executives
    
    4. INVESTMENT OUTLOOK:
       - Key strengths and risks
       - Overall sentiment (bullish/neutral/bearish)
    
    Provide specific numbers and dates.
    """
    
    print(f"📊 Analyzing {company_name} ({ticker})...\n")
    result = agent_executor.invoke({"input": query})
    
    return {
        "ticker": ticker,
        "company": company_name,
        "timestamp": datetime.now().isoformat(),
        "analysis": result["output"]
    }

# Test with a hot stock
analysis = analyze_earnings("META", "Meta Platforms")
print("\n" + "="*60)
print(f"📈 {analysis['company']} Analysis Complete")
print("="*60)
print(analysis['analysis'])

## Batch Analysis: Portfolio Scanner

Analyze multiple stocks at once!

In [ ]:
# Define your portfolio
portfolio = [
    ("GOOGL", "Alphabet"),
    ("AMZN", "Amazon"),
    ("TSLA", "Tesla")
]

print("🎯 Running portfolio analysis...\n")

portfolio_results = []
for ticker, name in portfolio:
    query = f"Get {name} ({ticker}) latest revenue and profit margin. Is it profitable?"
    
    print(f"Analyzing {name}...")
    result = agent_executor.invoke({"input": query})
    
    portfolio_results.append({
        "ticker": ticker,
        "company": name,
        "summary": result["output"][:200] + "..."  # Truncate for display
    })
    print(f"✅ {name} complete\n")

# Display results
print("\n" + "="*60)
print("📊 PORTFOLIO SUMMARY")
print("="*60)
for result in portfolio_results:
    print(f"\n{result['company']} ({result['ticker']}):")
    print(result['summary'])

## Advanced: Custom Analysis Pipeline

Build your own financial analysis workflow!

In [ ]:
class EarningsAnalyzer:
    """
    Automated earnings analysis pipeline.
    """
    
    def __init__(self, agent_executor):
        self.agent = agent_executor
    
    def quick_metrics(self, ticker: str) -> dict:
        """Get quick financial metrics."""
        query = f"Get {ticker} latest revenue, net income, and total assets. Just the numbers."
        result = self.agent.invoke({"input": query})
        return {"ticker": ticker, "metrics": result["output"]}
    
    def insider_sentiment(self, ticker: str) -> str:
        """Analyze insider trading sentiment."""
        query = f"Are {ticker} insiders buying or selling? Last 30 days. One sentence summary."
        result = self.agent.invoke({"input": query})
        return result["output"]
    
    def risk_assessment(self, ticker: str) -> str:
        """Identify key risks."""
        query = f"What are the top 3 risks for {ticker} based on recent filings?"
        result = self.agent.invoke({"input": query})
        return result["output"]
    
    def full_analysis(self, ticker: str, company: str) -> dict:
        """Complete earnings analysis."""
        print(f"\n🔍 Starting full analysis for {company} ({ticker})...\n")
        
        # Get all components
        metrics = self.quick_metrics(ticker)
        insider = self.insider_sentiment(ticker)
        risks = self.risk_assessment(ticker)
        
        return {
            "company": company,
            "ticker": ticker,
            "financial_metrics": metrics["metrics"],
            "insider_activity": insider,
            "key_risks": risks,
            "analyzed_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

# Initialize analyzer
analyzer = EarningsAnalyzer(agent_executor)

# Run full analysis
result = analyzer.full_analysis("AAPL", "Apple")

print("\n" + "="*60)
print("📱 APPLE COMPREHENSIVE ANALYSIS")
print("="*60)
print(f"\n📊 Financial Metrics:\n{result['financial_metrics']}")
print(f"\n💼 Insider Activity:\n{result['insider_activity']}")
print(f"\n⚠️ Key Risks:\n{result['key_risks']}")
print(f"\n⏰ Analyzed: {result['analyzed_at']}")

## What's Next?

### Ideas to Extend This:

1. **Slack Bot**: Get real-time filing alerts
2. **Dashboard**: Build a Streamlit app with live metrics
3. **Trading Signals**: Combine with technical analysis
4. **PDF Reports**: Generate professional analysis reports
5. **Screening**: Find stocks meeting specific criteria

### Resources:

- GitHub: [sec-edgar-agentkit](https://github.com/stefanoamorelli/sec-edgar-agentkit)
- LangChain Docs: [python.langchain.com](https://python.langchain.com)
- SEC EDGAR: [sec.gov/edgar](https://www.sec.gov/edgar)

### Installation:

```bash
pip install sec-edgar-langchain
```

---

**Built for the financial AI community** 🚀